Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.7173687384069132, θ=0.13269804264364593)  …  Distributions.Gamma(α=0.6204217061574602, θ=0.458661340786815)  
 Distributions.Gamma(α=0.028181280157484112, θ=0.574443713743908)     Distributions.Gamma(α=0.1913771929914556, θ=0.22530770518147158)
 Distributions.Gamma(α=0.4925727068335153, θ=0.3385610244382222)      Distributions.Gamma(α=0.7781809217838502, θ=0.24181305057566682)
 Distributions.Gamma(α=0.36858795350830786, θ=0.6552631755824636)     Distributions.Gamma(α=0.7691396702272097, θ=0.6227295831535526) 
 Distributions.Gamma(α=0.7866182800526011, θ=0.5638663855554993)      Distributions.Gamma(α=0.656072614293141, θ=0.25216500765095384) )

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.730082  0.187215  0.473082   0.753872
 0.83677   0.894245  0.0278103  0.520485
 0.807294  0.470275  0.287478   0.134075)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7733"),[:i,:j],3x4 Array{Float64,2}:
 0.189825  0.589654  0.0637865  0.284751 
 0.151476  0.174609  0.104013   0.0121357
 0.882755  0.812735  0.976195   0.332184 )

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,3,5],Dict(:a=>[1,2],:b=>[2,3]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.8772290159796459, σ=0.5233160580742797)    …  Distributions.Normal(μ=0.46147530599420183, σ=0.4817106421487254)
 Distributions.Normal(μ=0.36274940750131734, σ=0.3625378041259075)      Distributions.Normal(μ=0.5977336508553064, σ=0.07912952931660544)
 Distributions.Normal(μ=0.20792728699737406, σ=0.16477166352282757)     Distributions.Normal(μ=0.6028150112069881, σ=0.5873109536670349) 
 Distributions.Normal(μ=0.06283042773333558, σ=0.595361598986397)       Distributions.Normal(μ=0.28098735396260377, σ=0.6023263345294803)
 Distributions.Normal(μ=0.35703364117355285, σ=0.03772399253939307)     Distributions.Normal(μ=0.6987265789483412, σ=0.8574152100287735) )

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.751465  0.214966  0.0279364  0.254294
 0.814174  0.470719  0.207949   0.577387
 0.973404  0.085687  0.38713    0.679353)

# Make a factor

In [8]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;

# Make a (generated) value function

In [9]:
value(f)

-34.223623859949456

In [10]:
g = @factor EntropyFactor x
value(g)

28.378770664093462

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [11]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [12]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1])),Dict(:τ=>:x,:b0=>:β,:a0=>:α))

In [13]:
value(obs), value(μ_prior), value(τ_prior)

(-169.54616636749657,-13.593072740907873,0.0449816833123928)

In [14]:
m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,[:i,:j],20x6 Array{Float64,2}:
 0.505511   0.752059    0.282409   0.0859192  0.782239    0.528333 
 0.797145   0.780359    0.50581    0.781343   0.288489    0.134635 
 0.898277   0.406798    0.0488168  

In [15]:
[n.name for n in m.nodes]

7-element Array{Any,1}:
 :μ 
 :τ 
 :μ0
 :τ0
 :a0
 :b0
 :y 

In [16]:
[typeof(f) for f in m.factors]

3-element Array{DataType,1}:
 VB.LogNormalFactor{2}
 VB.LogNormalFactor{1}
 VB.LogGammaFactor{1} 

In [17]:
m.graph

Dict{VB.Node,Array{Tuple{VB.Factor{N},Symbol},1}} with 7 entries:
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…

In [18]:
naturals(obs, μ)

6-element Array{Tuple{Float64,Float64},1}:
 (12.64378189096637,-11.000000000000002) 
 (10.50918712643105,-11.000000000000002) 
 (9.027984416721933,-11.000000000000002) 
 (9.11265189368914,-11.000000000000002)  
 (8.316503083015334,-11.000000000000002) 
 (10.083696731233921,-11.000000000000002)

In [19]:
naturals(obs, τ)

6-element Array{Tuple{Float64,Float64},1}:
 (10.0,14.168568950237152)
 (10.0,12.984869853439548)
 (10.0,12.50869560118426) 
 (10.0,12.339505246753953)
 (10.0,12.045626866883062)
 (10.0,12.951677852047776)

In [20]:
check_conjugate(τ, m)

true

In [21]:
check_conjugate(μ, m)

true

In [22]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)

In [23]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)

In [24]:
update!(m)

In [25]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=1.162329321893965, σ=0.7071067811865476)  
 Distributions.Normal(μ=-0.3470571111942618, σ=0.7071067811865476)
 Distributions.Normal(μ=-1.3944255915414672, σ=0.7071067811865476)
 Distributions.Normal(μ=-1.3345566444319994, σ=0.7071067811865476)
 Distributions.Normal(μ=-1.8975188672930523, σ=0.7071067811865476)
 Distributions.Normal(μ=-0.6479242549678952, σ=0.7071067811865476)

In [26]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=20.318446719670966, θ=0.8999999999999999)
 Distributions.Gamma(α=23.505072715107417, θ=0.8999999999999999)
 Distributions.Gamma(α=49.3973342780189, θ=0.8999999999999999)  
 Distributions.Gamma(α=47.20569246701965, θ=0.8999999999999999) 
 Distributions.Gamma(α=68.39751584778, θ=0.8999999999999999)    
 Distributions.Gamma(α=29.089255973720203, θ=0.8999999999999999)